In [1]:
import cv2
import os
import re

Cap = cv2.VideoCapture('Night Time Traffic Camera video.mp4')

BS_KNN = cv2.createBackgroundSubtractorKNN()
BS_MOG2 = cv2.createBackgroundSubtractorMOG2()

vehicles = 0
cont_valid = []
while Cap.isOpened():
    ret, frame = Cap.read() 

    # extract the foreground mask
    fgMask = BS_KNN.apply(frame)
    
    cv2.line(frame, (0,480), (1300,480), (0,0,255), 2) # Line
    cv2.line(frame, (0,470), (1300,470), (0,255,0), 1) # Offset ABOVE
    cv2.line(frame, (0,490), (1300,490), (0,255,0), 1) # Offset BELOW
    
    contours, _ = cv2.findContours(fgMask, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    
    for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        
        cont = (w > 50) and (h > 50)
        if not cont:
            continue
        
        if x > 20 and x < 1300 and y > 100:
            
            cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 2)
            xMid = int((x + (x+w))/2)
            yMid = int((y + (y+h))/2)
            cv2.circle(frame, (xMid,yMid),5,(0,0,255),5)

            cont_valid.append((xMid,yMid))

            for (X, Y) in cont_valid:
                if Y > 475 and Y < 485 : 
                    vehicles += 1
                    cont_valid.remove((X,Y))
                    
                  
    cv2.imshow('Foreground Mask', fgMask)
    cv2.putText(frame, 'Total vehicles : {}'.format(vehicles), (480, 30), cv2.FONT_HERSHEY_PLAIN, 2, (0,255,255), 2)
    cv2.imshow('Original Video', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# release video capture
cv2.destroyAllWindows()
Cap.release()